In [18]:
# Static network
import dynet as dy

# multilayer perceptron with a single hidden layer
# input 2 nodes
# hidden layer: 8 nodes, activation: tanh
# output layer: 1 node
# σ(V(tanh(Wx+b)))
# x: 2x1
# W: 8x2
# b: 8 vector
# V: 8x1

# define the parameters
m = dy.ParameterCollection()
pW = m.add_parameters((8,2)) # _dynet.Parameters
pV = m.add_parameters((1,8))
pb = m.add_parameters((8))

In [19]:
dy.renew_cg() # new computation graph

In [20]:
# add the parameters to the graph
# create Expression representing the network 
#(the network will include the Expressions 
# for the Parameters defined in the parameter collection)
W = dy.parameter(pW) #W is of type _dynet.Expression
V = dy.parameter(pV)
b = dy.parameter(pb)

In [21]:
# create the network
x = dy.vecInput(2) # an input vector of size 2. _dynet._vecInputExpression
output = dy.logistic(V*(dy.tanh((W*x)+b))) # output is _dynet.Expression

In [19]:
print(type(x))
print(type(output))

<class '_dynet._vecInputExpression'>
<class '_dynet.Expression'>


In [22]:
# we can now query our network
x.set([0,0])
output.value()

0.46384352445602417

In [23]:
# we want to be able to define a loss, so we need an input expression to work against.
y = dy.scalarInput(0) # this will hold the correct answer
loss = dy.binary_log_loss(output, y)

In [18]:
print(type(y))
print(type(loss))

<class '_dynet._inputExpression'>
<class '_dynet.Expression'>


In [24]:
trainer = dy.SimpleSGDTrainer(m)

In [21]:
type(trainer)

_dynet.SimpleSGDTrainer

In [23]:
# single step optimization/training

x.set([1,0])
y.set(1)
loss_value = loss.value() # this performs a forward through the network.
print("the loss before step is:",loss_value)

# now do an optimization step
loss.backward()  # compute the gradients
trainer.update()

# see how it affected the loss:
loss_value = loss.value(recalculate=True) # recalculate=True means "don't use precomputed value"
print("the loss after step is:",loss_value)

the loss before step is: 0.31683480739593506
the loss after step is: 0.289972722530365


In [25]:
# create training data set
def create_xor_instances(num_rounds=2000):
    questions = []
    answers = []
    for round in range(num_rounds):
        for x1 in 0,1:
            for x2 in 0,1:
                answer = 0 if x1==x2 else 1
                questions.append((x1,x2))
                answers.append(answer)
    return questions, answers

questions, answers = create_xor_instances()

In [28]:
print(len(questions))
print(len(answers))
print(questions[:4])
print(answers[:4])

8000
8000
[(0, 0), (0, 1), (1, 0), (1, 1)]
[0, 1, 1, 0]


In [29]:
list(zip(questions, answers))[:4]

[((0, 0), 0), ((0, 1), 1), ((1, 0), 1), ((1, 1), 0)]

In [26]:
# training
total_loss = 0
seen_instances = 0
for question, answer in zip(questions, answers):
    x.set(question)
    y.set(answer)
    seen_instances += 1
    total_loss += loss.value() # forward
    loss.backward()
    trainer.update()
    if (seen_instances > 1 and seen_instances % 100 == 0):
        print("average loss is:",total_loss / seen_instances)

average loss is: 0.7381698548793793
average loss is: 0.7264014571905136
average loss is: 0.7161895747979482
average loss is: 0.6998325595259667
average loss is: 0.6645818127989769
average loss is: 0.6091600617766381
average loss is: 0.5500666074880531
average loss is: 0.49701920384541154
average loss is: 0.45168140908496246
average loss is: 0.4132660636696964
average loss is: 0.38058659604157913
average loss is: 0.3525680349774969
average loss is: 0.328335457962866
average loss is: 0.30719786852066006
average loss is: 0.288612498573338
average loss is: 0.2721513508545468
average loss is: 0.25747434265856795
average loss is: 0.24430880694469023
average loss is: 0.23243408323878326
average loss is: 0.22166996149439364
average loss is: 0.21186797930925552
average loss is: 0.20290481876878239
average loss is: 0.19467725546686385
average loss is: 0.18709826192430531
average loss is: 0.1800939725853503
average loss is: 0.17360130059094026
average loss is: 0.1675660497346824
average loss is: 

In [34]:
b

expression 2/1

In [27]:
x.set([0,1])
print("0,1",output.value())

x.set([1,0])
print("1,0",output.value())

x.set([0,0])
print("0,0",output.value())

x.set([1,1])
print("1,1",output.value())

0,1 0.9973587989807129
1,0 0.9973693490028381
0,0 0.0009477359708398581
1,1 0.0028826759662479162


In [28]:
W.value()

array([[-2.79832745,  3.65569925],
       [ 2.30283499,  2.28838158],
       [-0.58401084, -0.34022495],
       [-0.8143546 , -0.74301219],
       [ 0.3137587 ,  0.31108665],
       [ 0.08065908,  0.21146144],
       [-0.26102978,  0.01117267],
       [ 3.64261055, -2.79860806]])

In [29]:
V.value()

array([[-5.4248147 ,  4.13615608,  1.02193916,  1.61990416, -1.33724082,
        -0.76941073,  0.27476746, -5.27176952]])

In [30]:
b.value()

[1.178240418434143,
 -0.4626430869102478,
 0.8583874106407166,
 1.3025976419448853,
 -1.0519248247146606,
 -0.6726115345954895,
 0.27763113379478455,
 1.1828503608703613]

In [31]:
m.save('models/xor_static_model')